In [2]:
%%capture
import numpy as np
import pandas as pd 

# Installing Hugginface sentence transformers
!pip install sentence_transformers          # to load SBert models from Huggingface repository
from sentence_transformers import SentenceTransformer, util

# Install Huggingface Transformers models and gradio
!pip install transformers
!pip install gradio
from transformers import pipeline # to load in configuration and weights of fine-tuned model
import gradio as gr # to create UI

In [5]:
# Loading back in our dataFrame
df = pd.read_json("krishnamurti_df.json")

In [6]:
%%capture
# Loading a pre-trained asymmetric semantic search model
model = SentenceTransformer("msmarco-roberta-base-v3") # best performing model

In [7]:
def get_similar_quotes(question):
  question_embedding = model.encode(question)
  sims = [util.dot_score(question_embedding, quote_embedding) for quote_embedding in df['Embedding']]
  ind = np.argpartition(sims, -5)[-5:]
  similar_sentences = [df['Quotes'][i] for i in ind]
  top5quotes = pd.DataFrame(data = similar_sentences, columns=["Quotes"], index=range(1,6))
  return top5quotes


In [8]:
demo = gr.Interface(fn=get_similar_quotes, 
                    title="Question2Quote",
                    description="Displays top 5 most related quotes to a given quesiton",
                    inputs=[gr.Textbox(
                                        placeholder="Ask question here!", 
                                        lines=5, 
                                        max_lines=10,
                                        label="Question")],
                    
                    outputs=[gr.DataFrame(
                        headers=["Quotes"],
                        max_rows=5,
                        interactive=False,
                        wrap=True)]
                    )

In [9]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://49330.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f4cd49cef50>,
 'http://127.0.0.1:7860/',
 'https://49330.gradio.app')